In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from CinePred.new_model import *
from CinePred.data.preprocessing import *

In [3]:
df = import_data('../raw_data/IMDb movies.csv')

In [11]:
df['budget'] = convert_budget_column(df[['budget']])

TypeError: 'float' object is not subscriptable

In [4]:
df.sort_values(by='budget', inplace=True)
df[df['budget'] == ''.head(50)

83176        $ 0
73034        $ 0
75333        $ 0
65543        $ 0
83047        $ 0
77523        $ 0
77145        $ 0
74996        $ 0
77212        $ 0
85342        $ 0
82903        $ 0
55022        $ 0
79408        $ 1
81215        $ 1
72514        $ 1
79383        $ 1
68277      $ 100
49104     $ 1000
69199     $ 1000
69183     $ 1000
77961     $ 1000
82864     $ 1000
79974     $ 1000
81629     $ 1000
50115     $ 1000
10952     $ 1000
67809     $ 1000
83995     $ 1000
53275     $ 1000
85658     $ 1000
82590     $ 1000
78198     $ 1000
73856     $ 1000
64899     $ 1000
64820     $ 1000
50722     $ 1000
31050     $ 1000
63688     $ 1000
54302     $ 1000
17454    $ 10000
63205    $ 10000
51402    $ 10000
58793    $ 10000
84586    $ 10000
2642     $ 10000
63083    $ 10000
15688    $ 10000
68736    $ 10000
38       $ 10000
72353    $ 10000
Name: budget, dtype: object

In [ ]:
df = preproc(df)

In [18]:
print(type(df.shape[0] / 2))
mid = int(df.shape[0] / 2)
print(mid)

<class 'float'>
6381


In [19]:
df.sort_values(by='budget', inplace=True)
df.reset_index(inplace=True)

df.drop(columns='index', inplace=True)

In [20]:
df1 = df.iloc[:mid]
df2 = df.iloc[mid:]

In [21]:
print(df1.shape)
print(df2.shape)

(6381, 28)
(6381, 28)


In [22]:
from sklearn.model_selection import cross_val_score

In [26]:
def get_mae2(df):
    X = df.drop(columns=['worlwide_gross_income'])
    y = df['worlwide_gross_income']
    model = XGBRegressor()
    print(cross_val_score(model, X, y, cv=5))

In [27]:
get_mae2(df1)

[-0.31820655  0.01651192 -0.05121392  0.01898145 -0.23328855]


In [28]:
get_mae2(df2)

[-0.18335718 -0.08749438  0.01314635 -0.05189867 -1.07857028]
